In [1]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import DynamicBayesianNetwork as DBN
from pgmpy.inference import DBNInference

In [2]:
# 实例化贝叶斯网络并添加网络节点和CPD
dbnet = DBN()
dbnet.add_edges_from([(('Z', 0), ('X', 0)), (('X', 0), ('Y', 0)),
                      (('Z', 0), ('Z', 1))])

z_start_cpd = TabularCPD(('Z', 0), 2, [[0.5], [0.5]])
x_i_cpd = TabularCPD(('X', 0), 2, [[0.6, 0.9],
                                   [0.4, 0.1]],
                     evidence=[('Z', 0)],
                     evidence_card=[2])
y_i_cpd = TabularCPD(('Y', 0), 2, [[0.2, 0.3],
                                   [0.8, 0.7]],
                     evidence=[('X', 0)],
                     evidence_card=[2])
z_trans_cpd = TabularCPD(('Z', 1), 2, [[0.7, 0.4],
                                       [0.3, 0.6]],
                         evidence=[('Z', 0)],
                         evidence_card=[2])
dbnet.add_cpds(z_start_cpd, z_trans_cpd, x_i_cpd, y_i_cpd)

In [3]:
dbnet.get_cpds()  # 此时的条件概率只有四个节点的！

[<TabularCPD representing P(('Z', 0):2) at 0x1c714be9040>,
 <TabularCPD representing P(('Y', 0):2 | ('X', 0):2) at 0x1c714be9070>,
 <TabularCPD representing P(('X', 0):2 | ('Z', 0):2) at 0x1c714bcee80>,
 <TabularCPD representing P(('Z', 1):2 | ('Z', 0):2) at 0x1c714be94f0>]

In [4]:
# 网络初始化，为了将 网络中隐藏的边和概率都补齐
dbnet.initialize_initial_state()
dbn_inf = DBNInference(dbnet)

In [5]:
dbnet.get_cpds()  # 初始化的网络，具有完整的6个节点的CPD

[<TabularCPD representing P(('Z', 0):2) at 0x1c714be9040>,
 <TabularCPD representing P(('Y', 0):2 | ('X', 0):2) at 0x1c714be9070>,
 <TabularCPD representing P(('X', 0):2 | ('Z', 0):2) at 0x1c714bcee80>,
 <TabularCPD representing P(('Z', 1):2 | ('Z', 0):2) at 0x1c714be94f0>,
 <TabularCPD representing P((Y, 1):2 | (X, 1):2) at 0x1c714bce2e0>,
 <TabularCPD representing P((X, 1):2 | (Z, 1):2) at 0x1c714bc2c10>]

# 此处为（"Z", 1）人工设置的先验概率
#### 先验概率也可以通过数据进行学习

In [6]:
print(dbnet.get_cpds()[3])

+-------------+-------------+-------------+
| ('Z', 0)    | ('Z', 0)(0) | ('Z', 0)(1) |
+-------------+-------------+-------------+
| ('Z', 1)(0) | 0.7         | 0.4         |
+-------------+-------------+-------------+
| ('Z', 1)(1) | 0.3         | 0.6         |
+-------------+-------------+-------------+


#  {('X', 1):0, ('Y', 1):1}条件下，（"Z", 1）的后验概率

In [7]:
print(dbn_inf.query([('Z', 1)], {('X', 1):0, ('Y', 1):1})[('Z', 1)])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

+-------------+-----------------+
| ('Z', 1)    |   phi(('Z', 1)) |
+=============+=================+
| ('Z', 1)(0) |          0.4490 |
+-------------+-----------------+
| ('Z', 1)(1) |          0.5510 |
+-------------+-----------------+


#  {('X', 1):0, ('Y', 1):1, ('Z', 0):0} 条件下，（"Z", 1）的后验概率————明先增大（"Z", 1）取 0 的概率

In [8]:
print(dbn_inf.query([('Z', 1)], {('X', 1):0, ('Y', 1):1, ('Z', 0):0})[('Z', 1)])

0it [00:00, ?it/s]

0it [00:00, ?it/s]

+-------------+-----------------+
| ('Z', 1)    |   phi(('Z', 1)) |
+=============+=================+
| ('Z', 1)(0) |          0.6087 |
+-------------+-----------------+
| ('Z', 1)(1) |          0.3913 |
+-------------+-----------------+


#  {('X', 1):0, ('Y', 1):1, ('Z', 0):1} 条件下，（"Z", 1）的后验概率————明先增大（"Z", 1）取 1 的概率

In [9]:
print(dbn_inf.query([('Z', 1)], {('X', 1):0, ('Y', 1):1, ('Z', 0):1})[('Z', 1)])

0it [00:00, ?it/s]

0it [00:00, ?it/s]

+-------------+-----------------+
| ('Z', 1)    |   phi(('Z', 1)) |
+=============+=================+
| ('Z', 1)(0) |          0.3077 |
+-------------+-----------------+
| ('Z', 1)(1) |          0.6923 |
+-------------+-----------------+


# 以上就是基于先验概率模型，再得到观测节点信息后，节点概率变化的直观展示